In [2]:
import requests
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# URL to make requests: fill in x and y coordinates
URL = 'http://65.108.84.176:8080/{}/{}'

# Task

### Visually explore the testing environment


In [3]:
def make_api_call(x: float = 0, y: float = 0) -> float:
    """Make API call to the testing server."""
    # Plug in x and y coordinates provided
    response = requests.get(URL.format(x, y))
    # Parse JSON with received z-coordinate
    return response.json()['z']    

#### Numerical representation of the environment

In [15]:
for y in reversed(range(-5, 5 + 1)):
    print('', end='\n')
    for x in range(-5, 5 + 1):
        z = str(make_api_call(x, y))
        z = ' ' + z if len(z) == 1 else z
        z = ' ' + z if len(z) == 2 else z
        z = ' ' + z if len(z) == 3 else z
        print(z, end='')

#### Single-step heatmap of the environment

In [51]:
def get_square_map(depth: int = 2, step: float = 1, name = 'test') -> np.ndarray:
    """Query a rectangular map representation of the environment."""
    size = 2 * depth + 1
    square_map = np.zeros((size, size))
    # Iterate over rows and columns
    desc = f'Downloading \"{name}\" {size}x{size} sample'
    for y in tqdm(reversed(range(-depth, depth + 1)), total=size, desc=desc):
        for x in range(-depth, depth + 1):
            square_map[-y + depth, x + depth] = make_api_call(x * step, y * step)
    # Save the map as a text file
    np.savetxt(f'{name}_map.txt', square_map, fmt='%d')
    return square_map
# Test
small_map_name = 'smöll'
small_map = get_square_map(depth=2, name=small_map_name)
small_map

In [52]:
def plot_heatmap(square_map: np.ndarray, name = 'test') -> None:
    """Plot the squared environment as a heatmap."""
    # Find min-max values to set up temperature properly
    min_val = np.min(square_map)
    max_val = np.max(square_map)
    abs_max = abs(min_val) if abs(min_val) > max_val else max_val
    # Plot heatmap
    plt.figure(figsize=(8, 6))
    plt.imshow(square_map, cmap='coolwarm', vmin=-abs_max, vmax=abs_max, interpolation='nearest')
    plt.title(f'The glorious {name} map')
    plt.colorbar()
    plt.show()
# Test
plot_heatmap(small_map, name=small_map_name)

In [53]:
map_81_name = 'mid'
map_81 = get_square_map(depth=4, name=map_81_name)
plot_heatmap(map_81, name=map_81_name)

In [54]:
map_361_name = 'biggie'
map_361 = get_square_map(depth=8, name=map_361_name)
plot_heatmap(map_361, name=map_361_name)

In [59]:
print(f'Maximum: {make_api_call(100, 100)}')
print(f'Minimum: {make_api_call(-100, -100)}')

* The environment contains z-values spanning from negative infinity to positive infinity.
* The environment gradually transitions between lows and highs:
    * global minimum at (8, 8)
    * global maximum at (-8, -8)
* The single-step map revealed only one local maximum and only one local minimum
* Revealing 100x100 map by single step takes too many explorations

---

#### Far-step heatmap of the environment

In [65]:
sizes = [
    (6, 16, '16_step_13_x_13'),
    (5, 20, '20_step_11_x_11'),
    (4, 25, '25_step_9_x_9'),
    (3, 33, '33_step_7_x_7'),
    (2, 50, '50_step_5_x_5'),
    (1, 100, '100_step_3_x_3'),
]
for depth, step, name in sizes:
    square_map = get_square_map(depth=depth, step=step, name=name)
    plot_heatmap(square_map, name=name)

* Revealing the 14x14 map would take 196 explorations.
* Revealing the 13x13 map with 16-long steps would take 169 explorations and cover -96 to 96 range.
* Revealing the 11x11 map with 20-long steps would take 121 explorations and cover -100 to 100 range.
* Revealing the 9x9 map with 25-long steps would take 81 explorations and cover -100 to 100 range.
* Revealing the 7x7 map with 33-long steps would take 49 explorations and cover -99 to 99 range.
* Map's z-values do span from negative infinity to positive infinity.
* The global maximum is located in (x=negative infinity, y=positive infitity).
* The global minimum is located in (x=positive infinity, y=negative infitity).
* The assumption could be made that there are no local minimums and maximums except the global ones.

---

#### Conclusions

1. The map is built using a linear function.
2. The linear function goes through zero-coordinates.
3. The best solution is at the edge of the map.
4. The linear regression should be the most optimal solution to learn the pattern.